In [6]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings('ignore')
 
path = ''
train_sales= pd.read_csv(path+'../ccf_car/train_sales_data.csv')#, engine='python')
train_search_data = pd.read_csv(path+'../ccf_car/train_search_data.csv')#, engine='python')
train_user_reply_data = pd.read_csv(path+'../ccf_car/train_user_reply_data.csv')#, engine='python')
test = pd.read_csv(path+'../ccf_car/evaluation_public.csv')#, engine='python')
 
# train_sales_data\train_search_data\train_user_reply_data  拼接
data = pd.merge(train_sales, train_search_data, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = pd.merge(data, train_user_reply_data, 'left', on=['model', 'regYear', 'regMonth'])
# 去异常
# 0.58973169000
def quantile_clip(group):
    #group.plot()
    group[group < group.quantile(.05)] = group.quantile(.05)
    group[group > group.quantile(.9)] = group.quantile(.9)
    #group.plot()
    #plt.show()
    return group
data['salesVolume'] = data.groupby(['adcode', 'model'])['salesVolume'].transform(quantile_clip)
def quantile_clip(group):
    #group.plot()
    group[group < group.quantile(.05)] = group.quantile(.05)
    group[group > group.quantile(.9)] = group.quantile(.95)
    #group.plot()
    #plt.show()
    return group
data['popularity'] = data.groupby(['adcode', 'model'])['popularity'].transform(quantile_clip)
data['carCommentVolum'] = data.groupby(['adcode', 'model'])['popularity'].transform(quantile_clip)
data['newsReplyVolum'] = data.groupby(['adcode', 'model'])['popularity'].transform(quantile_clip)
 
# 统计销量
data['bt_ry_mean'] = data.groupby(['bodyType','regYear'])['salesVolume'].transform('mean')
data['ad_ry_mean'] = data.groupby(['adcode','regYear'])['salesVolume'].transform('mean')
data['md_ry_mean'] = data.groupby(['model','regYear'])['salesVolume'].transform('mean')
 
 
'''
一、lgb预测
'''
# 测试集并入
data = pd.concat([data, test], ignore_index=True)
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['forecastVolum']
# 填补测试集的车身类型
data['bodyType'] = data['model'].map(train_sales.drop_duplicates('model').set_index('model')['bodyType'])
# 编码 bodyType、model
for i in ['bodyType', 'model']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))
# 距离2016年的时间间隔，月数
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']
 
shift_feat = []
data['model_adcode'] = data['adcode'] + data['model']
data['model_adcode_mt'] = data['model_adcode'] * 100 + data['mt']

# 填充测试集特征值
'''for col in ['carCommentVolum','newsReplyVolum','popularity','bt_ry_mean','ad_ry_mean', 'md_ry_mean']:
    #1月
    m1_12 = data.loc[(data.regYear==2017)&(data.regMonth==1) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==12), col].values
    m1_11 = data.loc[(data.regYear==2017)&(data.regMonth==1) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==11), col].values
    m1_10 = data.loc[(data.regYear==2017)&(data.regMonth==1) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==10), col].values
    m1_09 = data.loc[(data.regYear==2017)&(data.regMonth==1) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==9) , col].values
    m1_08 = data.loc[(data.regYear==2017)&(data.regMonth==1) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==8) , col].values
    m1_07 = data.loc[(data.regYear==2017)&(data.regMonth==1) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==7) , col].values
    m1_06 = data.loc[(data.regYear==2017)&(data.regMonth==1) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==6) , col].values

    m1_12_volum = data.loc[(data.regYear==2017)&(data.regMonth==12), col].values * m1_12
    m1_11_volum = data.loc[(data.regYear==2017)&(data.regMonth==11), col].values * m1_11
    m1_10_volum = data.loc[(data.regYear==2017)&(data.regMonth==10), col].values * m1_10
    m1_09_volum = data.loc[(data.regYear==2017)&(data.regMonth==9) , col].values * m1_09
    m1_08_volum = data.loc[(data.regYear==2017)&(data.regMonth==8) , col].values * m1_08
    m1_07_volum = data.loc[(data.regYear==2017)&(data.regMonth==7) , col].values * m1_07
    m1_06_volum = data.loc[(data.regYear==2017)&(data.regMonth==6) , col].values * m1_06

    data.loc[(data.regMonth==1)&(data.regYear==2018), col] = m1_12_volum/2 + m1_11_volum/4 + m1_10_volum/8 + m1_09_volum/16 + m1_08_volum/32 + m1_07_volum/64 + m1_06_volum/64

    #2月
    m2_01 = data.loc[(data.regYear==2017)&(data.regMonth==2) , col].values / data.loc[(data.regYear==2017)&(data.regMonth==1 ) , col].values
    m2_12 = data.loc[(data.regYear==2017)&(data.regMonth==2) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==12) , col].values
    m2_11 = data.loc[(data.regYear==2017)&(data.regMonth==2) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==11) , col].values
    m2_10 = data.loc[(data.regYear==2017)&(data.regMonth==2) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==10) , col].values
    m2_09 = data.loc[(data.regYear==2017)&(data.regMonth==2) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==9 ) , col].values
    m2_08 = data.loc[(data.regYear==2017)&(data.regMonth==2) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==8 ) , col].values
    m2_07 = data.loc[(data.regYear==2017)&(data.regMonth==2) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==7 ) , col].values

    m2_01_volum = data.loc[(data.regYear==2018)&(data.regMonth==1 ), col].values * m2_01
    m2_12_volum = data.loc[(data.regYear==2017)&(data.regMonth==12), col].values * m2_12
    m2_11_volum = data.loc[(data.regYear==2017)&(data.regMonth==11), col].values * m2_11
    m2_10_volum = data.loc[(data.regYear==2017)&(data.regMonth==10), col].values * m2_10
    m2_09_volum = data.loc[(data.regYear==2017)&(data.regMonth==9) , col].values * m2_09
    m2_08_volum = data.loc[(data.regYear==2017)&(data.regMonth==8) , col].values * m2_08
    m2_07_volum = data.loc[(data.regYear==2017)&(data.regMonth==7) , col].values * m2_07

    data.loc[(data.regMonth==2)&(data.regYear==2018), col] = m2_01_volum/2 + m2_12_volum/4 + m2_11_volum/8 + m2_10_volum/16 + m2_09_volum/32 + m2_08_volum/64 + m2_07_volum/64

    #3月
    m3_02 = data.loc[(data.regYear==2017)&(data.regMonth==3) , col].values / data.loc[(data.regYear==2017)&(data.regMonth==2 ) , col].values
    m3_01 = data.loc[(data.regYear==2017)&(data.regMonth==3) , col].values / data.loc[(data.regYear==2017)&(data.regMonth==1 ) , col].values
    m3_12 = data.loc[(data.regYear==2017)&(data.regMonth==3) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==12) , col].values
    m3_11 = data.loc[(data.regYear==2017)&(data.regMonth==3) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==11) , col].values
    m3_10 = data.loc[(data.regYear==2017)&(data.regMonth==3) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==10) , col].values
    m3_09 = data.loc[(data.regYear==2017)&(data.regMonth==3) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==9 ) , col].values
    m3_08 = data.loc[(data.regYear==2017)&(data.regMonth==3) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==8 ) , col].values

    m3_02_volum = data.loc[(data.regYear==2018)&(data.regMonth==2 ), col].values * m3_02
    m3_01_volum = data.loc[(data.regYear==2018)&(data.regMonth==1 ), col].values * m3_01
    m3_12_volum = data.loc[(data.regYear==2017)&(data.regMonth==12), col].values * m3_12
    m3_11_volum = data.loc[(data.regYear==2017)&(data.regMonth==11), col].values * m3_11
    m3_10_volum = data.loc[(data.regYear==2017)&(data.regMonth==10), col].values * m3_10
    m3_09_volum = data.loc[(data.regYear==2017)&(data.regMonth==9) , col].values * m3_09
    m3_08_volum = data.loc[(data.regYear==2017)&(data.regMonth==8) , col].values * m3_08

    data.loc[(data.regMonth==3)&(data.regYear==2018), col] = m3_02_volum/2 + m3_01_volum/4 + m3_12_volum/8 + m3_11_volum/16 + m3_10_volum/32 + m3_09_volum/64 + m3_08_volum/64

    #4月
    m4_03 = data.loc[(data.regYear==2017)&(data.regMonth==4) , col].values / data.loc[(data.regYear==2017)&(data.regMonth==3 ) , col].values
    m4_02 = data.loc[(data.regYear==2017)&(data.regMonth==4) , col].values / data.loc[(data.regYear==2017)&(data.regMonth==2 ) , col].values
    m4_01 = data.loc[(data.regYear==2017)&(data.regMonth==4) , col].values / data.loc[(data.regYear==2017)&(data.regMonth==1 ) , col].values
    m4_12 = data.loc[(data.regYear==2017)&(data.regMonth==4) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==12) , col].values
    m4_11 = data.loc[(data.regYear==2017)&(data.regMonth==4) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==11) , col].values
    m4_10 = data.loc[(data.regYear==2017)&(data.regMonth==4) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==10) , col].values
    m4_09 = data.loc[(data.regYear==2017)&(data.regMonth==4) , col].values / data.loc[(data.regYear==2016)&(data.regMonth==9 ) , col].values
    
    m4_03_volum = data.loc[(data.regYear==2018)&(data.regMonth==3 ), col].values * m4_03
    m4_02_volum = data.loc[(data.regYear==2018)&(data.regMonth==2 ), col].values * m4_02
    m4_01_volum = data.loc[(data.regYear==2018)&(data.regMonth==1 ), col].values * m4_01
    m4_12_volum = data.loc[(data.regYear==2017)&(data.regMonth==12), col].values * m4_12
    m4_11_volum = data.loc[(data.regYear==2017)&(data.regMonth==11), col].values * m4_11
    m4_10_volum = data.loc[(data.regYear==2017)&(data.regMonth==10), col].values * m4_10
    m4_09_volum = data.loc[(data.regYear==2017)&(data.regMonth==9 ), col].values * m4_09

    data.loc[(data.regMonth==4)&(data.regYear==2018), col] = m4_03_volum/2 + m4_02_volum/4 + m4_01_volum/8 + m4_12_volum/16 + m4_11_volum/32 + m3_10_volum/64 + m4_09_volum/64
'''
for col in ['carCommentVolum','newsReplyVolum','popularity','bt_ry_mean','ad_ry_mean', 'md_ry_mean']:
    lgb_col_na = pd.isnull(data[col])
    data[col] = data[col].replace(0,1)
    data.loc[lgb_col_na,col] = \
    ((((data.loc[(data['regYear'].isin([2017]))&(data['regMonth'].isin([1,2,3,4])), col].values /
    data.loc[(data['regYear'].isin([2016]))&(data['regMonth'].isin([1,2,3,4])), col].values)))*
    data.loc[(data['regYear'].isin([2017]))&(data['regMonth'].isin([1,2,3,4])), col].values * 1.03).round()
 

 
# 每年的新年在第几月份
data['happyNY'] = 0
data.loc[(data['regYear'].isin([2016,2018])&data['regMonth'].isin([2])),'happyNY'] = 1
data.loc[(data['regYear'].isin([2017])&data['regMonth'].isin([1])),'happyNY'] = 1
 
 
# label 下移12个月，则测试集填充上了label
for i in [4]:
    shift_feat.append('shift_model_adcode_mt_label_{0}'.format(i))
    data['model_adcode_mt_{0}'.format(i)] = data['model_adcode_mt'] + i
    data_last = data[~data.label.isnull()].set_index('model_adcode_mt_{0}'.format(i))
    data['shift_model_adcode_mt_label_{0}'.format(i)] = data['model_adcode_mt'].map(data_last['label'])
 
data.loc[pd.isnull(data['shift_model_adcode_mt_label_4']),'shift_model_adcode_mt_label_4'] = \
((data.loc[(data.regMonth.isin([1,2,3,4]))&(data.regYear.isin([2016])),'label'].values/
 data.loc[(data.regMonth.isin([1,2,3,4]))&(data.regYear.isin([2017])),'label'].values)*
data.loc[(data.regMonth.isin([1,2,3,4]))&(data.regYear.isin([2016])),'label'].values).round()
 
# 根据月份添加权重值
a = 6; b = 4
data['weightMonth'] = data['regMonth'].map({1:a, 2:a, 3:a, 4:a,
                                            5:b, 6:b, 7:b, 8:b, 9:b, 10:b, 11:b, 12:b,})
 
 
 
def score(data):
    pred = data.groupby(['adcode', 'model'])['pred_label'].agg(lambda x: list(x))
    label = data.groupby(['adcode', 'model'])['label'].agg(lambda x: list(x))
    label_mean = data.groupby(['adcode', 'model'])['label'].agg(lambda x: np.mean(x))
    data_agg = pd.DataFrame()
    data_agg['pred_label'] = pred
    data_agg['label'] = label
    data_agg['label_mean'] = label_mean
    nrmse_score = []
    for raw in data_agg.values:
        nrmse_score.append(mse(raw[0], raw[1]) ** 0.5 / raw[2])
    return 1 - np.mean(nrmse_score)
 
 
df_lgb = pd.DataFrame({'id': test['id']})
for col_add in ['ad_ry_mean', 'md_ry_mean', 'bt_ry_mean']:
    # 取用的字段，用于训练模型
    num_feat = shift_feat
    cate_feat = ['adcode', 'bodyType', 'model', 'regYear', 'regMonth', 'happyNY']
    features = num_feat + cate_feat + ['popularity', 'carCommentVolum', 'newsReplyVolum', 'weightMonth'] + [col_add]  # [ad_ry_mean, md_ry_mean, bt_ry_mean]
 
    train_idx = (data['mt'] <= 20) # 小于等于20月以内的数据作为训练集
    valid_idx = (data['mt'].between(21, 24)) # 21到24个月的数据作为验证集
    test_idx = (data['mt'] > 24) # 大于24个月的是测试集
 
    # label
    data['n_label'] = np.log(data['label'])
 
    train_x = data[train_idx][features]
    train_y = data[train_idx]['n_label']
 
    valid_x = data[valid_idx][features]
    valid_y = data[valid_idx]['n_label']
 
    ############################### lgb ###################################
    lgb_model = lgb.LGBMRegressor(
        num_leaves=40, reg_alpha=1, reg_lambda=0.1, objective='mse',
        max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=2019,
        n_estimators=8000, subsample=0.8, colsample_bytree=0.8)
 
    lgb_model.fit(train_x, train_y, eval_set=[(valid_x, valid_y)],
                  categorical_feature=cate_feat, early_stopping_rounds=100, verbose=300)
    data['pred_label'] = np.e ** lgb_model.predict(data[features])
    model = lgb_model
    # 特征重要程度
    print ('lgb特征重要程度：',sorted(dict(zip(train_x.columns,model.feature_importances_)).items(),key=lambda x: x[1], reverse=True))
    print('NRMSE的均值:',score(data = data[valid_idx]))
    model.n_estimators = model.best_iteration_
    model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)
    data['forecastVolum'] = np.e ** model.predict(data[features])
    sub = data[test_idx][['id']]
    sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    sub_lgb = sub.reset_index(drop=True)
    sub_lgb = sub_lgb[['id','forecastVolum']]
    print('lgb中forecastVolmn的0值数量：',(sub_lgb['forecastVolum']==0).sum())
    df_lgb[col_add] = sub_lgb['forecastVolum']
df_lgb.to_csv("../rst/057baseline_rid_abnormal.csv", index=False) 
# df_lgb有三列值，任一一列提交，上0.57，祝各位好运！！

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[67]	valid_0's l2: 0.11934
lgb特征重要程度： [('model', 742), ('shift_model_adcode_mt_label_4', 523), ('regMonth', 466), ('adcode', 254), ('regYear', 163), ('ad_ry_mean', 118), ('carCommentVolum', 94), ('popularity', 90), ('weightMonth', 89), ('bodyType', 44), ('happyNY', 22), ('newsReplyVolum', 8)]
NRMSE的均值: 0.7232201727271007
lgb中forecastVolmn的0值数量： 0
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[67]	valid_0's l2: 0.120495
lgb特征重要程度： [('model', 690), ('shift_model_adcode_mt_label_4', 495), ('regMonth', 452), ('adcode', 320), ('md_ry_mean', 188), ('regYear', 139), ('weightMonth', 99), ('carCommentVolum', 98), ('popularity', 85), ('bodyType', 19), ('happyNY', 19), ('newsReplyVolum', 9)]
NRMSE的均值: 0.7197283110413955
lgb中forecastVolmn的0值数量： 0
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	valid